In [2]:
import os
import json
import pandas as pd
while os.getcwd() != '/home/jupyter/crisp':
    os.chdir("..")
%pwd

'/home/jupyter/crisp'

In [36]:
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from google.cloud import storage


In [5]:
data = pd.read_pickle('data/cosmic_rads/docker_exps/all_human_mice_data_combined.pkl')
data.environment.unique()
environment_indices = data.environment
predictor_indices = data.target

top_1000_variance_columns = list(data.std().sort_values(ascending=False).iloc[:1000].index)
data = data[top_1000_variance_columns]
        

In [6]:
data['Target'] = predictor_indices
data['environment'] = environment_indices


In [26]:
mouse_samples = data[data['environment'] != 'Homo sapiensHBEC3KT']

In [27]:
human_samples = data[data['environment'] == 'Homo sapiensHBEC3KT']

In [29]:
from google.cloud import storage
storage_client = storage.Client()
bucket = storage_client.get_bucket('ah21_data')

In [41]:
experiment_directory = 'heavy_ion_exps_with_varying_humans/' 

In [42]:
for n in tqdm(range(0, len(human_samples), 3)):
    human_samples.sample(n)
    seed_directory = 'mice_41_humans_{}/'.format(str(n))
    data = pd.concat([mouse_samples, human_samples.sample(n)])
    print(data.environment.value_counts())
    for index, environment in enumerate(data.environment.unique()):
        colab_directory = 'col_{}/'.format(index)
        env_data = data[data.environment == environment]
        env_data = env_data.drop('environment', axis=1)
        train, test = train_test_split(env_data, test_size=0.2, random_state = 0)

        blob = bucket.blob(experiment_directory+seed_directory+colab_directory+'train/data.csv')
        train.to_csv('heavy_ion_data.csv')
        blob.upload_from_filename('heavy_ion_data.csv')

        blob = bucket.blob(experiment_directory+seed_directory+colab_directory+'test/data.csv')
        test.to_csv('heavy_ion_data.csv')
        blob.upload_from_filename('heavy_ion_data.csv')

  0%|          | 0/26 [00:00<?, ?it/s]

Mus musculusmale      22
Mus musculusfemale    19
Name: environment, dtype: int64
Mus musculusmale       22
Mus musculusfemale     19
Homo sapiensHBEC3KT     3
Name: environment, dtype: int64
Mus musculusmale       22
Mus musculusfemale     19
Homo sapiensHBEC3KT     6
Name: environment, dtype: int64
Mus musculusmale       22
Mus musculusfemale     19
Homo sapiensHBEC3KT     9
Name: environment, dtype: int64
Mus musculusmale       22
Mus musculusfemale     19
Homo sapiensHBEC3KT    12
Name: environment, dtype: int64
Mus musculusmale       22
Mus musculusfemale     19
Homo sapiensHBEC3KT    15
Name: environment, dtype: int64
Mus musculusmale       22
Mus musculusfemale     19
Homo sapiensHBEC3KT    18
Name: environment, dtype: int64
Mus musculusmale       22
Homo sapiensHBEC3KT    21
Mus musculusfemale     19
Name: environment, dtype: int64
Homo sapiensHBEC3KT    24
Mus musculusmale       22
Mus musculusfemale     19
Name: environment, dtype: int64
Homo sapiensHBEC3KT    27
Mus musculus

In [ ]:
    storage_client = storage.Client()
    bucket = storage_client.get_bucket('ah21_data')